In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import pymongo
from splinter import Browser
import requests
import time
import re

News

In [2]:
# set up chromedriver
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
#  go to mars nasa site
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [4]:
# Create soup object
html = browser.html
soup = bs(html, 'html.parser')
find_element = soup.select_one('ul.item_list li.slide')

In [5]:
# fine parent element
find_element.find("div", class_='content_title')

<div class="content_title"><a href="/news/8716/nasa-to-broadcast-mars-2020-perseverance-launch-prelaunch-activities/" target="_self">NASA to Broadcast Mars 2020 Perseverance Launch, Prelaunch Activities</a></div>

In [6]:
# Find news title
news_title = find_element.find("div", class_="content_title").get_text()
news_title

'NASA to Broadcast Mars 2020 Perseverance Launch, Prelaunch Activities'

In [7]:
# find news paragraph

news_p = find_element.find("div", class_="article_teaser_body").get_text()
news_p

'Starting July 27, news activities will cover everything from mission engineering and science to returning samples from Mars to, of course, the launch itself.'

Featured Image

In [8]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

In [9]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(url)

In [10]:
# find an click image on webpage

find_elem2 = browser.find_by_id('full_image')
find_elem2.click()


In [11]:
# find & click more info

browser.is_element_present_by_text('more info', wait_time=1)
find_elem3 = browser.find_link_by_partial_text('more info')
find_elem3.click()

C:\Users\jacob\Anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:528: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


In [12]:
html = browser.html
soup = bs(html, 'html.parser')

In [13]:
# find image url

image_url = soup.select_one('figure.lede a img').get("src")
image_url

'/spaceimages/images/largesize/PIA18840_hires.jpg'

In [14]:
featured_image_url  = "https://www.jpl.nasa.gov" + image_url
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18840_hires.jpg


Weather

In [15]:
url = "https://twitter.com/marswxreport?lang=en"
browser.visit(url)

time.sleep(5)

In [16]:
html = browser.html
soup = bs(html, 'html.parser')

In [17]:
# locate tweet refering to Mars Weather
mars_tweet = soup.find('div', attrs={"class": "tweet", "data-name": "Mars Weather"})

In [18]:
#   search for tweet with p/span tag with class tweet-text
    try:
        weather = mars_tweet.find('p', "tweet-text").get_text()
        weather
        
    except AttributeError:
        pattern = re.compile(r'sol')
        weather = soup.find('span', text=pattern).text
        weather
        
    weather

'InSight sol 585 (2020-07-19) low -89.2ºC (-128.5ºF) high -12.0ºC (10.3ºF)\nwinds from the W at 6.2 m/s (13.9 mph) gusting to 18.9 m/s (42.3 mph)\npressure at 7.80 hPa'

# Hemispheres

In [19]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)


In [20]:
# create empty list for urls
hemisphere_image_urls = []

# find image urls
urls = browser.find_by_css("a.product-item h3")

for i in range(len(urls)):
    hemisphere = {}
    
    browser.find_by_css("a.product-item h3")[i].click()
    
    find_element3 = browser.find_link_by_text('Sample').first
    hemisphere['img_url']= find_element3['href']
    
    hemisphere['title'] = browser.find_by_css("h2.title").text
    
    hemisphere_image_urls.append(hemisphere)
    
    browser.back()

C:\Users\jacob\Anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:536: FutureWarning: browser.find_link_by_text is deprecated. Use browser.links.find_by_text instead.
  FutureWarning,


In [21]:
hemisphere

{'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
 'title': 'Valles Marineris Hemisphere Enhanced'}

In [22]:
hemisphere_image_urls

[{'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

# Facts

In [24]:
# Create dataframe with facts then convert to html

mars_df = pd.read_html('https://space-facts.com/mars/')[0]
mars_df.columns = ["Question", "Answer"]
mars_df


,Question,Answer
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [25]:
mars_df.set_index('Question', inplace=True)
mars_df

,Answer
Question,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [27]:
mars_df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Answer</th>\n    </tr>\n    <tr>\n      <th>Question</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n 